# Property-based Testing mit Hypothesis

Simon Jakobi (https://github.com/sjakobi)

http://hypothesis.readthedocs.org

## Properties

* Hält meine Funktion für alle gültigen Inputs einen ordentlichen Output aus?
* gilt `decode(encode(x)) == x`?

In [136]:
from hypothesis import given
import hypothesis.strategies as st

@given(st.integers())
def test_int_str_conversion(n):
    assert int(str(n)) == n + 3
    
test_int_str_conversion()

Falsifying example: test_int_str_conversion(n=0)


AssertionError: 

In [147]:
from hypothesis import Settings, assume, Verbosity, example
from math import isnan

@example(-0.0)
@example()
@given(st.floats(), settings=Settings(verbosity=Verbosity.verbose))
def test_float_str_conversion(n):
    assume(not isnan(n))
    assert float(str(n)) == n
    
test_float_str_conversion()

Trying example: test_float_str_conversion(n=nan)
Trying example: test_float_str_conversion(n=2.1718898336961224e-308)
Trying example: test_float_str_conversion(n=2.439002582286872e+171)
Trying example: test_float_str_conversion(n=2.43343449057803e-309)
Trying example: test_float_str_conversion(n=9.224387598352012e+194)
Trying example: test_float_str_conversion(n=-5.19742018467483e+198)
Trying example: test_float_str_conversion(n=1.1509345063172101e+61)
Trying example: test_float_str_conversion(n=-1.1423116146569761e+191)
Trying example: test_float_str_conversion(n=9.052443705885417e-25)
Trying example: test_float_str_conversion(n=1.21122104621569e+41)
Trying example: test_float_str_conversion(n=-7.57954637549207e-109)
Trying example: test_float_str_conversion(n=6.0)
Trying example: test_float_str_conversion(n=32.0)
Trying example: test_float_str_conversion(n=9.0)
Trying example: test_float_str_conversion(n=11.0)
Trying example: test_float_str_conversion(n=1.0)
Trying example: test_floa

In [153]:
def quicksort(lst):
    if not lst:
        return []
    lst_ = lst[:]
    x = lst_.pop()
    return (quicksort([y for y in lst_ if y <= x]) +
            [x] +
            quicksort([y for y in lst_ if y >= x]))

In [ ]:
sorted(sorted(xs)) == sorted(xs)

In [154]:
quicksort([0])

[0]

In [155]:
quicksort(quicksort([0]))b

[0]

In [161]:
quicksort([0])

[0]

In [158]:
@given(st.lists(st.integers(), max_size=3)) # settings=Settings(verbosity=Verbosity.verbose))
def test_quicksort_is_idempotent(lst):
    sorted_ = quicksort(lst)
    assert quicksort(sorted_) == sorted_
    
test_quicksort_is_idempotent()

Falsifying example: test_quicksort_is_idempotent(lst=[0, 0])


AssertionError: 

In [200]:
@given(st.lists(st.integers(), max_size=10),
       st.tuples(st.integers(min_value=0, max_value=10), st.integers(min_value=0, max_value=10)))
def test_quicksort_result_is_sorted(lst, indices):
    assume(indices[0] < len(lst))
    assume(indices[1] < len(lst))
    idx1, idx2 = sorted(indices)
    sorted_lst = quicksort(lst)
    assert sorted_lst[idx1] <= sorted_lst[idx2]
    
test_quicksort_result_is_sorted()


In [162]:
@given(st.lists(st.integers()))
def test_compare_to_sorted(lst):
    assert quicksort(lst) == sorted(lst)
    
test_compare_to_sorted()


Falsifying example: test_compare_to_sorted(lst=[0, 0])


AssertionError: 

In [163]:
import pytest

help(pytest)

Help on module pytest:

NAME
    pytest - pytest: unit and functional testing with Python.

SUBMODULES
    collect

CLASSES
    builtins.Exception(builtins.BaseException)
        _pytest.config.UsageError
    builtins.object
        _pytest.config.cmdline
    
    class UsageError(builtins.Exception)
     |  error in pytest usage or invocation
     |  
     |  Method resolution order:
     |      UsageError
     |      builtins.Exception
     |      builtins.BaseException
     |      builtins.object
     |  
     |  Data descriptors defined here:
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)
     |  
     |  ----------------------------------------------------------------------
     |  Methods inherited from builtins.Exception:
     |  
     |  __init__(self, /, *args, **kwargs)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  __new__(*args, **kwargs) from builtins.type
     |      Create and retur

## Strategies

In [8]:
help(st.integers)

Help on function integers in module hypothesis.strategies:

integers(min_value=None, max_value=None)
    Returns a strategy which generates integers (in Python 2 these may be
    ints or longs).
    
    If min_value is not None then all values will be >=
    min_value. If max_value is not None then all values will be <= max_value



In [33]:
st.integers().map(lambda n: 2 * n).example()

-16

In [170]:
st.one_of(st.integers(), st.floats()).example()

3.404632732869028e-45

In [132]:
help(st.lists)

Help on function lists in module hypothesis.strategies:

lists(elements=None, min_size=None, average_size=None, max_size=None, unique_by=None)
    Returns a list containining values drawn from elements length in the
    interval [min_size, max_size] (no bounds in that direction if these are
    None). If max_size is 0 then elements may be None and only the empty list
    will be drawn.
    
    average_size may be used as a size hint to roughly control the size
    of list but it may not be the actual average of sizes you get, due
    to a variety of factors.
    
    if unique_by is not None it must be a function returning a hashable type
    when given a value drawn from elements. The resulting list will satisfy the
    condition that for i != j, unique_by(result[i]) != unique_by(result[j]).



In [133]:
help(st.recursive)

Help on function recursive in module hypothesis.strategies:

recursive(base, extend, max_leaves=100)
    base: A strategy to start from
    extend: A function which takes a strategy and returns a new strategy
    max_leaves: The maximum number of elements to be drawn from base on a given
    run.
    
    This returns a strategy S such that S = extend(base | S). That is, values
    maybe drawn from base, or from any strategy reachable by mixing
    applications of | and extend.
    
    An example may clarify: recursive(booleans(), lists) would return a
    strategy that may return arbitrarily nested and mixed lists of booleans.
    So e.g. False, [True], [False, []], [[[[True]]]], are all valid values to
    be drawn from that strategy.



## Eigene Datenstrukturen

In [201]:
from collections import namedtuple
from math import sqrt

from hypothesis import Settings, Verbosity

Point = namedtuple("Point", "x, y")

def distance(p1, p2):
    dx = (p1.x - p2.x) ** 2
    dy = (p1.y - p2.y) ** 2
    return sqrt(dx + dy)

points = st.builds(Point, x=st.floats(), y=st.floats())

In [171]:
# Ein konventioneller Test für die Plausibilität
def test_distance_from_0_0_to_0_1_is_1():
    assert distance(Point(0,0), Point(0,1)) == 1

test_distance_from_0_0_to_0_1_is_1()

In [202]:
@given(points)
def test_distance_for_same_point_is_zero(point):
    assert distance(point, point) == 0
    
test_distance_for_same_point_is_zero()

Falsifying example: test_distance_for_same_point_is_zero(point=Point(x=0.0, y=inf))


AssertionError: 

In [173]:
@given(points, points)
def test_distance_is_always_non_negative(p1, p2):
    assert distance(p1, p2) >= 0
    
test_distance_is_always_non_negative()

In [174]:
@given(points, points)
def test_distance_is_commutative(p1, p2):
    assert distance(p1, p2) == distance(p2, p1)
    
test_distance_is_commutative()

In [ ]:
#@given(points, points, settings=Settings(verbosity=Verbosity.verbose))

In [175]:
help(st.streaming)

Help on function streaming in module hypothesis.strategies:

streaming(elements)
    Generates an infinite stream of values where each value is drawn from
    elements.
    
    The result is iterable (the iterator will never terminate) and
    indexable.



In [190]:
from math import pi

def hyperbel(n):
    return 1 / (n + pi)

In [193]:
@given(st.floats())
def test_hyperbel(n):
    assert isinstance(hyperbel(n), object)
    
test_hyperbel()

In [194]:
pi + (-pi) == 0

True

## Vorteile von Property-based Testing

* Kann Sonderfälle finden, auf die man selbst nicht so schnell gekommen wäre
* Viele Testcases mit wenig Code
* Regt an, über Spezifikationen nachzudenken

## Nachteile von Property-based Testing

* Geeignete Properties sind nicht immer leicht zu finden
* Testdurchläufe können lange dauern
* Tests können kompliziert werden, gar selbst testbedürftig werden

In [206]:
from pprint import pprint

strat = st.text()
pprint([strat.example() for _ in range(20)])

['k\U000a0509¹¹ꃥ¹\U000a0509\x8bꃥ\x8bk\U000a050922>>¹>k',
 '\U000484ae\x11\n'
 '\x82\U000484ae\x82\x11\x82\n'
 '\t\x82\x82\x06\n'
 '\x1e'
 '\U000484ae\x11\n'
 '\x1e'
 '\U000484ae\x1e'
 '\x06\x11\x1e'
 '\x11\x06\x1e',
 '\n-\x91ě\x91ěěě¸',
 '\x7f?Şl\x7fŦćŞćŶ\x7f',
 '\x8f\x8f\x8f\x8f\x8f',
 '\x91\x91\x07\x91\x91\x07\x07\x07\x91\x07\x07\x07\x07\x07\x07\x91\x07\x91\x07\x91\x07\x07\x07\x91\x07\x91\x91\x91\x07\x07\x91\x07\x07',
 'BĂ',
 'ęę\t',
 '\x8d\x00',
 '\x05\x05\x05\x05\x05\x05\x05\x05\x05\x05\x05\x05\x05\x05\x05\x05',
 'k\x9a5ŇȌȌȌõkõkŇȌŇȌ5k5\x9a\x0b'
 ' ŇõŇȌ5k\x0b'
 '\x0b'
 'k\x9a555õ\x9aŇ\x9aõkkŇ 5k5õ\x0b'
 '5Ȍ\x9a\x0b'
 ' õ\x9a5',
 'DĭLL¾L\x02\x8e\x11pĭ\x11D0p\x01j\x02\x8ej¾\x01Lĭ¾\x8e',
 ':\\',
 '\x1c'
 '\x1f_\U000ee21a\u2002\x10_L\x1f\x16\U000ae9dd\x1f\U000ee21a\U000ee21a\u2006',
 'Ķ\x82ȠȠN\x02_&×\x8aF&YMT\x82]\x02YNTȠ\x82',
 '\x04\x86\x86\x04¥\x8d\x8d\x8d\x04\x04T\x8d¥T¥¥M\x8d\x86\x86¥\x86\x04',
 '2@@¬J@JħJ\x0bJ))¬J\x84\x84ħ\x84ħ',
 '{\x0fi\U000a4f6d{¥\x1a\x03\U00037b09\x88g\U00037b

## Weiterführendes

Apis testen:
    
* http://wildfish.com/blog/2015/10/01/using-gabbi-and-hypothesis-test-django-apis/
* http://hypothesis.readthedocs.org/en/latest/examples.html#fuzzing-an-http-api

Eine Testsuite vom Hypothesis-Autor selbst:

https://github.com/DRMacIver/intset/blob/master/tests/test_intset.py

In [ ]:
version_number = st.builds(lambda a, b, c: "{}.{}.{}", st.integers.)

In [207]:
print(bool(str(False)))

True


In [217]:
import json

bool(json.loads("1"))

True

In [ ]:
st.floats(min_value=1, max_value=3).filter(lambda x: x != 3)

In [220]:
help(st.floats().flatmap)d

Help on method flatmap in module hypothesis.searchstrategy.strategies:

flatmap(expand) method of hypothesis.searchstrategy.reprwrapper.ReprWrapperStrategy instance
    Returns a new strategy that generates values by generating a value
    from this strategy, say x, then generating a value from
    strategy(expand(x))
    
    This method is part of the public API.



In [247]:
[st.integers(min_value=0, max_value=4).flatmap(lambda x: st.lists(st.integers(), min_size=x, max_size=x)).example()  for _ in range(10)]

[[],
 [1, 1],
 [156],
 [],
 [7626991385634418447858669401108046134297458164580397149127436020566383995],
 [-40498937442166127291724863202403824045,
  -40498748695777628408447093307083567649,
  -40498850174745270443052997023427733557],
 [],
 [],
 [82650778004, 25472172698, 414623347],
 [32]]

In [248]:
help(st.lists)

Help on function lists in module hypothesis.strategies:

lists(elements=None, min_size=None, average_size=None, max_size=None, unique_by=None)
    Returns a list containining values drawn from elements length in the
    interval [min_size, max_size] (no bounds in that direction if these are
    None). If max_size is 0 then elements may be None and only the empty list
    will be drawn.
    
    average_size may be used as a size hint to roughly control the size
    of list but it may not be the actual average of sizes you get, due
    to a variety of factors.
    
    if unique_by is not None it must be a function returning a hashable type
    when given a value drawn from elements. The resulting list will satisfy the
    condition that for i != j, unique_by(result[i]) != unique_by(result[j]).



In [250]:
my_strings = st.sampled_from("a b c d e f".split())

strings

In [256]:
short_list_of_strings = st.lists(my_strings, max_size=5)

short_list_of_strings.example()

['e']

In [249]:
help(st.sampled_from)

Help on function sampled_from in module hypothesis.strategies:

sampled_from(elements)
    Returns a strategy which generates any value present in the iterable
    elements.
    
    Note that as with just, values will not be copied and thus you
    should be careful of using mutable data

